In [57]:
#importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [58]:
#load the dataframe
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')


In [59]:
#drop unnecessary culumns
target_train = train_data['SalePrice']
train1 = train_data.drop(['Id','SalePrice'],axis=1)
test1 = test_data.drop(['Id'],axis=1)

#combine train and test datasets to impute missng values at once
#axis = 0(append the rows), 1(append to the columns)

all_data = pd.concat([train1,test1], axis=0)
all_data['MSSubClass'] =  all_data['MSSubClass'].astype(str)

In [60]:
#all_data.select_dtypes(np.number) ----see columns with nuerical data
all_data.select_dtypes('object').loc[:,all_data.isnull().sum()>0].columns   #cat-columns with NaN 

Index(['MSZoning', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Electrical', 'KitchenQual', 'Functional',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PoolQC', 'Fence', 'MiscFeature', 'SaleType'],
      dtype='object')

In [61]:
all_data2 = all_data.copy()
#cat_col1 - NA means a feature,  cat_col2---Na -truly missing
cat_col1 = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType','GarageFinish', 'GarageQual','GarageCond','PoolQC',
       'Fence','MiscFeature']
cat_col2 = ['MSZoning','Utilities','Exterior1st', 'Exterior2nd', 'MasVnrType','Electrical', 'KitchenQual','Functional','SaleType'] 
for columns in cat_col1:
    all_data2[columns] = all_data2[columns].fillna('None')
for columns in cat_col2:
    all_data2[columns] = all_data2[columns].fillna(all_data2[columns].mode()[0])


<strong> df['feature1'].mode(): The .mode() function is used to calculate the mode (most frequent value) of the 'feature1' column. It returns a pandas Series object containing the mode(s). [0]: The [0] index is used to access the first (or only) element of the mode Series. Since the mode function returns the mode(s) in descending order, [0] selects the most frequent value. <strong>Because<strong/>

<strong>Because<strong/>

In [62]:
all_data2['MSZoning'].mode()

0    RL
Name: MSZoning, dtype: object

In [68]:
##imputing missing numeric values##

def Knn_Imputer(all_data2,na_target):
    all_data2=all_data2.copy()
    num_df = all_data2.select_dtypes(np.number)
    avail_col = num_df.loc[:,num_df.isnull().sum()==0].columns

    x_train = num_df.loc[num_df[na_target].isnull()== False, avail_col]
    y_train = num_df.loc[num_df[na_target].isnull()== False, na_target]
    x_test =num_df.loc[num_df[na_target].isnull()== True,avail_col]
    
    knn = KNeighborsRegressor()
    knn.fit(x_train,y_train)
    y_pred = knn.predict(x_test)
    
    all_data2.loc[all_data2[na_target].isnull()==True  , na_target] = y_pred
    
    return all_data2




In [69]:
all_data2.columns[all_data2.isnull().sum() > 0]

Index(['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
       'GarageCars', 'GarageArea'],
      dtype='object')

In [70]:
for na_col in ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
       'GarageCars', 'GarageArea']:
    all_data2 = Knn_Imputer(all_data2,na_col)

In [71]:
all_data2.isna().sum()

MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
KitchenQual 

In [74]:
clean_data = all_data2.copy